In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-10-13 2025-10-12


In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
#yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-10-13
yesterday: 2025-10-10 00:00:00


In [3]:
yesterday = yesterday.date()
a_year_ago = yesterday - timedelta(days=365)
print(yesterday, a_year_ago)

2025-10-10 2024-10-10


### Restart and Run All Cells

In [5]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [6]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Weekly
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### Weekly process or when stataus changes.

In [8]:
pd.read_sql_query('SELECT COUNT(*) AS records FROM sales', conlite)

,records
0,1842


In [9]:
sqlDel = text("""DELETE FROM sales""")
rp = conlite.execute(sqlDel)
conlite.commit()
rp.rowcount

1842

In [10]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df_orders = pd.read_sql(sql, conlite)

names = df_orders['name'].values.tolist()
portfolio = ", ".join(map(lambda name: "'%s'" % name, names))
portfolio

"'AIMIRT', 'CPF', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'MCS', 'NER', 'ORI', 'PTG', 'PTT', 'RCL', 'SYNEX', 'TFFIF', 'TVO', 'WHAIR', 'WHART'"

In [11]:
len(portfolio.split(','))

18

### Get past one year data

In [13]:
sql = """
    SELECT name, date, price, qty, maxp, minp 
    FROM price 
    WHERE date >= '{}' AND name IN ({}) 
    ORDER BY name, date
""".format(a_year_ago, portfolio)
print(sql)


    SELECT name, date, price, qty, maxp, minp 
    FROM price 
    WHERE date >= '2024-10-10' AND name IN ('AIMIRT', 'CPF', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'MCS', 'NER', 'ORI', 'PTG', 'PTT', 'RCL', 'SYNEX', 'TFFIF', 'TVO', 'WHAIR', 'WHART') 
    ORDER BY name, date



In [14]:
df_price = pd.read_sql(sql, const)
df_price.tail()

,name,date,price,qty,maxp,minp
4331,WHART,2025-10-06,9.70,1910524,9.80,9.65
4332,WHART,2025-10-07,9.75,1335553,9.80,9.70
4333,WHART,2025-10-08,9.70,1772431,9.75,9.65
4334,WHART,2025-10-09,9.70,391581,9.75,9.65
4335,WHART,2025-10-10,9.65,511715,9.75,9.60


In [15]:
file_name = "Yearly-Price-by-Name.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\Yearly-Price-by-Name.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Yearly-Price-by-Name.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Yearly-Price-by-Name.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Yearly-Price-by-Name.csv


In [16]:
df_price.set_index("name", inplace=True)
df_price.to_csv(output_file, header=None)
df_price.to_csv(god_file, header=None)
df_price.to_csv(icd_file, header=None)
df_price.to_csv(osd_file, header=None)

### Create monitors from orders

In [18]:
sql = """
    SELECT name,trade 
    FROM orders 
    WHERE name IN ({})
    ORDER BY name
""".format(portfolio)
df_orders = pd.read_sql(sql, conlite)
df_orders.set_index(['name'],inplace=True)
df_orders

,trade
name,
AIMIRT,B
CPF,B
DIF,S
GVREIT,B
IVL,B
JMART,B
JMT,B
MCS,S
NER,B


In [19]:
file_name = "monitors.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [20]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\monitors.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\monitors.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\monitors.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\monitors.csv


In [21]:
df_orders.to_csv(output_file, header=None)
df_orders.to_csv(god_file)
df_orders.to_csv(icd_file)
df_orders.to_csv(osd_file)

In [22]:
sql = """
SELECT trade, COUNT(*) AS items 
FROM orders
GROUP BY trade
ORDER BY trade
"""
grp = pd.read_sql(sql, conlite)
grp

,trade,items
0,B,11
1,S,7


### For new orders that never have XXX.html & XXX.CSV

In [24]:
file_name = 'price-uploads.csv'
input_file = os.path.join(dat_path, file_name)
print(input_file)

C:\Users\PC1\OneDrive\A5\Data\price-uploads.csv


In [25]:
uploads = pd.read_csv(input_file)
uploads.sort_values(['name'],ascending=[True]).shape

(152, 1)

In [26]:
df_merge = pd.merge(df_orders, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(152, 3)

In [27]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,trade,_merge


In [28]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(134, 3)

In [29]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
lite_stocks = pd.read_sql(sql, conlite)
lite_stocks.set_index(["name"], inplace=True)
lite_stocks.shape

(29, 2)

In [30]:
file_name = "stocks-all.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [31]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\stocks-all.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\stocks-all.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\stocks-all.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\stocks-all.csv


In [32]:
lite_stocks.to_csv(output_file, header=None)
lite_stocks.to_csv(god_file)
lite_stocks.to_csv(icd_file)
lite_stocks.to_csv(osd_file)

In [33]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
AIMIRT,B
CPF,B
GVREIT,B
IVL,B
JMART,B
JMT,B
NER,B
ORI,B
RCL,B


In [34]:
buy_candidates.shape[0]

11

In [35]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
DIF,S
MCS,S
PTG,S
PTT,S
SYNEX,S
TVO,S
WHART,S


In [36]:
sell_candidates.shape[0]

7

In [37]:
conlite.close()
const.close()

In [38]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-10-13 20:49:30
